# FashionMNIST Neural Network

In [19]:
%matplotlib inline

### Import the necessary libraries

In [35]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import Lambda, ToTensor
from matplotlib import pyplot as plt

### Download training data from open datasets

In [36]:
training_data = datasets.FashionMNIST(root='data',
                                      train=True,
                                      download=True,
                                      transform=ToTensor()) # download Fashion MNIST training dataset
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

### Download test data from open datasets

In [37]:
testing_data = datasets.FashionMNIST(root='data',
                                      train=False,
                                      download=True,
                                      transform=ToTensor()) # download Fashion MNIST testing dataset
testing_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [38]:
from torch.utils.data import DataLoader

train_dl = DataLoader(training_data,
                      batch_size = 64,
                      shuffle = True) # load partial data so we don't crash the system

test_dl = DataLoader(testing_data,
                      batch_size = 64,
                      shuffle = True) # load partial data so we don't crash the system


In [39]:
train_features, train_labels = next(iter(train_dl)) # this is if you want to look at the data  inside as you can't call train_dl directly
test_features, test_labels = next(iter(test_dl))

train_features.shape, train_labels.shape            # 64 rows, 1 col, 28 x 28 pixels
test_features.shape, test_labels.shape              # 64 rows

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

### Get cpu or gpu device for training

In [40]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' # if we have a gpu, we will use that. If not we will use cpu
device

'cuda'

### Define NN model

In [61]:
from torch import nn 

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.NN = nn.Sequential(nn.Flatten(),
                                nn.Linear(28*28, 450),
                                nn.LogSigmoid(),
                                nn.Linear(450, 100),
                                nn.Hardtanh(),
                                nn.Linear(100, 10),
                                nn.Hardswish())

    def forward(self, x):
        #logits = self.NN(x)                       # it's logits if the last layer is not an activation function, else its pred
        #pred = nn.Softmax(dim=1)(logits)
        pred = self.NN(x)
        return pred

model = NeuralNetwork().to(device) # create the model using the device (cpu/gpu)

### Optimizing the Model Parameters

In [52]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        X = X.to(device)      # use gpu by moving data to gpu
        y = y.to(device)
        
        # Compute prediction and loss
        preds = model(X)
        loss = loss_fn(preds, y)
        
        # Backpropagation
        optimizer.zero_grad() # set gradients to zero
        loss.backward()       # recalculate gradients with new values for weights and biases
        optimizer.step()      # adjust the weights and biases based on new gradients 

        if batch % 100 == 0:  # Book keeping to see our progress
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [53]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():        # disable autograds
        for X, y in dataloader:
            X = X.to(device)     # use gpu by moving data to gpu
            y = y.to(device)
            
            # Compute prediction, accuracy and loss
            pred = model(X)
            test_loss += loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [62]:
epochs = 25         
learning_rate = 1e-3 # 0.001

loss_fn = nn.CrossEntropyLoss()                                    # create loss function
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  # create optimizer

for t in range(epochs):                                            
    print(f"Epoch {t+1}\n-------------------------------")
    # train_loop
    train_loop(train_dl, model, loss_fn, optimizer)
    
    #test_loop
    test_loop(test_dl, model, loss_fn)
    
print("Done!")

Epoch 1
-------------------------------
loss: 2.304689  [    0/60000]
loss: 2.297739  [ 6400/60000]
loss: 2.284675  [12800/60000]
loss: 2.297423  [19200/60000]
loss: 2.282977  [25600/60000]
loss: 2.291730  [32000/60000]
loss: 2.280680  [38400/60000]
loss: 2.286123  [44800/60000]
loss: 2.276484  [51200/60000]
loss: 2.263857  [57600/60000]
Test Error: 
 Accuracy: 35.8%, Avg loss: 0.035647 

Epoch 2
-------------------------------
loss: 2.272923  [    0/60000]
loss: 2.273957  [ 6400/60000]
loss: 2.260088  [12800/60000]
loss: 2.252306  [19200/60000]
loss: 2.247666  [25600/60000]
loss: 2.247673  [32000/60000]
loss: 2.235411  [38400/60000]
loss: 2.225111  [44800/60000]
loss: 2.212979  [51200/60000]
loss: 2.197710  [57600/60000]
Test Error: 
 Accuracy: 42.3%, Avg loss: 0.034668 

Epoch 3
-------------------------------
loss: 2.196602  [    0/60000]
loss: 2.207886  [ 6400/60000]
loss: 2.193743  [12800/60000]
loss: 2.173895  [19200/60000]
loss: 2.138376  [25600/60000]
loss: 2.132745  [32000/600

loss: 0.669377  [ 6400/60000]
loss: 0.621105  [12800/60000]
loss: 0.749178  [19200/60000]
loss: 0.608201  [25600/60000]
loss: 0.571381  [32000/60000]
loss: 0.523625  [38400/60000]
loss: 0.919148  [44800/60000]
loss: 0.555771  [51200/60000]
loss: 0.557145  [57600/60000]
Test Error: 
 Accuracy: 76.1%, Avg loss: 0.010336 

Epoch 23
-------------------------------
loss: 0.612837  [    0/60000]
loss: 0.688603  [ 6400/60000]
loss: 0.621635  [12800/60000]
loss: 0.629625  [19200/60000]
loss: 0.601384  [25600/60000]
loss: 0.567466  [32000/60000]
loss: 0.864176  [38400/60000]
loss: 0.520633  [44800/60000]
loss: 0.677145  [51200/60000]
loss: 0.612717  [57600/60000]
Test Error: 
 Accuracy: 76.5%, Avg loss: 0.010200 

Epoch 24
-------------------------------
loss: 0.735146  [    0/60000]
loss: 0.614797  [ 6400/60000]
loss: 0.637342  [12800/60000]
loss: 0.492802  [19200/60000]
loss: 0.734672  [25600/60000]
loss: 0.718053  [32000/60000]
loss: 0.734683  [38400/60000]
loss: 0.586277  [44800/60000]
loss

### Saving Models

In [63]:
import torch
import torchvision.models as models

torch.save(model.state_dict(), 'model_parameters.pth') # save model parameters into a file

### Loading Models

In [64]:
loaded_model = model                                              # call our model that we created before
loaded_model.load_state_dict(torch.load('model_parameters.pth'))  # load the parameters from file to model
loaded_model.eval()                                               # sets self.train to false for certian hidden layers

NeuralNetwork(
  (NN): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=450, bias=True)
    (2): LogSigmoid()
    (3): Linear(in_features=450, out_features=100, bias=True)
    (4): Hardtanh(min_val=-1.0, max_val=1.0)
    (5): Linear(in_features=100, out_features=10, bias=True)
    (6): Hardswish()
  )
)

This model can now be used to make predictions.



In [65]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]                                                              # we define which class represent which object

model.eval()                                                   # sets self.train to false for certian hidden layers
x, y = test_features[0], test_labels[0]                        # choose the first random data point in data set
x, y = torch.tensor(x).to(device), torch.tensor(y).to(device)  # move data to gpu if used
with torch.no_grad():                                          # disable autograd
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y] # retrive predicted and actual clothing name
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Trouser", Actual: "Trouser"


<ipython-input-65-6d40e98a0f8d>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x, y = torch.tensor(x).to(device), torch.tensor(y).to(device)  # move data to gpu if used
